Get Tags List and User List

In [1]:
import pandas as pd
import copy

# Load the recipes and interactions data
recipes = pd.read_csv("RAW_recipes.csv")
interactions = pd.read_csv("RAW_interactions.csv")

# Extract unique tags from the dataset
unique_tags = set()
for tag_list in recipes['tags']:
    # Correcting parsing based on your description that tags appear like a Python list in string format
    tag_list = tag_list.strip('[]').replace("'", "").split(', ')
    for tag in tag_list:
        cleaned_tag = tag.strip()  # Remove any leading/trailing spaces
        if cleaned_tag:  # Ensure the tag is not empty
            unique_tags.add(cleaned_tag)

# Extract unique users from the dataset
users = interactions['user_id'].unique()

Make User Profiles

In [2]:
# Create an empty dictionary to store the tags for each user
user_tags = {user: {} for user in users}
number_appeared = {user: {} for user in users}

# Iterate over the interactions data to update the user_tags dictionary
for index, row in interactions.iterrows():
    user_id = row['user_id']
    recipe_id = row['recipe_id']
    rating = row['rating']
    recipe_tags = recipes[recipes['id'] == recipe_id]['tags'].values[0]
    recipe_tags = recipe_tags.strip('[]').replace("'", "").split(', ')
    for tag in recipe_tags:
        cleaned_tag = tag.strip()
        if cleaned_tag:
            if cleaned_tag not in user_tags[user_id]:
                user_tags[user_id][cleaned_tag] = 0
                number_appeared[user_id][cleaned_tag] = 0
            user_tags[user_id][cleaned_tag] += rating
            number_appeared[user_id][cleaned_tag] += 1

# Get Average rating for each tag
for user in user_tags:
    for tag in user_tags[user]:
        if number_appeared[user][tag] > 0:
            user_tags[user][tag] /= number_appeared[user][tag]


In [3]:
import recommender

group_preferences = []
for user in user_tags:
    user_preferences = {tag: rating for tag, rating in user_tags[user].items()}
    group_preferences.append(user_preferences)
    
recs = []

recs.append(recommender.average_recommendation(group_preferences[:5]))
print("AVG REC: ", recs[0])

recs.append(recommender.least_misery_recommendation(group_preferences[:5]))
print("LM REC: ", recs[1])

recs.append(recommender.most_pleasure_recommendation(group_preferences[:5]))
print("MP REC: ", recs[2])


AVG REC:  [('sole-and-flounder', 5.0), ('egyptian', 5.0), ('cuban', 5.0), ('pears', 5.0), ('libyan', 5.0), ('manicotti', 5.0), ('mardi-gras-carnival', 5.0), ('hawaiian', 5.0), ('oaxacan', 5.0), ('ontario', 5.0)]
LM REC:  [('sole-and-flounder', 5.0), ('egyptian', 5.0), ('cuban', 5.0), ('pears', 5.0), ('libyan', 5.0), ('manicotti', 5.0), ('mardi-gras-carnival', 5.0), ('hawaiian', 5.0), ('oaxacan', 5.0), ('ontario', 5.0)]
MP REC:  [('weeknight', 5.0), ('fruit', 5.0), ('vegan', 5.0), ('30-minutes-or-less', 5.0), ('low-protein', 5.0), ('salads', 5.0), ('dinner-party', 5.0), ('low-cholesterol', 5.0), ('low-carb', 5.0), ('healthy-2', 5.0)]


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Convert the recipe tags to a DataFrame
recipe_tag_matrix = []
recipe_ids = []
recipe_names = []

for index, row in recipes.iterrows():
    recipe_id = row['id']
    recipe_tags = row['tags'].strip('[]').replace("'", "").split(', ')
    recipe_tags = [tag.strip() for tag in recipe_tags if tag.strip()]
    tag_vector = {tag: 1 for tag in recipe_tags}
    recipe_tag_matrix.append(tag_vector)
    recipe_ids.append(recipe_id)
    recipe_names.append(row['name'])

recipe_tag_df = pd.DataFrame(recipe_tag_matrix).fillna(0)

# Compute similarity scores between group tags and recipes
def compute_similarity(group_vector, recipe_matrix):
    group_vector = np.array(group_vector).reshape(1, -1)
    similarity_scores = cosine_similarity(group_vector, recipe_matrix)
    return similarity_scores.flatten()

for recommendations in recs:
    recommendations = dict(recommendations)

    # Generate recommendations for the group based on similarity
    group_vector = [recommendations.get(tag, 0) for tag in recipe_tag_df.columns]
    similarity_scores = compute_similarity(group_vector, recipe_tag_df.values)
    recommended_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order
    recommended_recipes = [recipe_names[idx] for idx in recommended_indices]

    # Print top 10 recommendations for the group
    print(f"Group recommendations: {recommended_recipes[:10]}")
    print("#############################################")

Group recommendations: ['easy as pita bread', 'pecan pear tossed salad', 'pear and blue cheese puffs', 'loz  almond sweetmeats', 'poached pear delight', 'thoom   arabic garlic sauce', 'taco salad supreme', 'middle eastern pilaff', 'maple grilled pears with brown sugar and cinnamon', 'broccoli and wasabi dip']
#############################################
Group recommendations: ['easy as pita bread', 'pecan pear tossed salad', 'pear and blue cheese puffs', 'loz  almond sweetmeats', 'poached pear delight', 'thoom   arabic garlic sauce', 'taco salad supreme', 'middle eastern pilaff', 'maple grilled pears with brown sugar and cinnamon', 'broccoli and wasabi dip']
#############################################
Group recommendations: ['onion and orange salad', 'watermelon  cucumber and feta salad', 'easy balsamic asaparagus medley', 'roasted carrot and avocado salad', 'scandanavian apple salad', 'caramelized pear and toasted almond salad', 'cucumber and celery salad', 'apple   spinach salad',

LLM recommender

In [9]:
import recommender
import ast

recommendations = recommender.openai_recommendation(group_preferences[:5])
recommendations = ast.literal_eval(recommendations)
print("OPENAI REC: ", recommendations)

recommendations_dict = {tag: rating for tag, rating in recommendations}

# Generate recommendations for the group based on similarity
group_vector = [recommendations_dict.get(tag, 0) for tag in recipe_tag_df.columns]
similarity_scores = compute_similarity(group_vector, recipe_tag_df.values)
recommended_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order
recommended_recipes = [recipe_names[idx] for idx in recommended_indices]

# Print top 10 recommendations for the group
print(f"Group recommendations: {recommended_recipes[:10]}")

OPENAI REC:  [('breads', 5.0), ('soups-stews', 4.916666666666667), ('oven', 4.916666666666667), ('refrigerator', 5.0), ('stews', 5.0), ('beginner-cook', 5.0), ('main-dish', 4.913660130718954), ('portuguese', 5.0), ('drop-cookies', 5.0), ('lunch', 4.883333333333333)]
OPENAI REC:  [('breads', 5.0), ('soups-stews', 4.916666666666667), ('oven', 4.916666666666667), ('refrigerator', 5.0), ('stews', 5.0), ('beginner-cook', 5.0), ('main-dish', 4.913660130718954), ('portuguese', 5.0), ('drop-cookies', 5.0), ('lunch', 4.883333333333333)]
Group recommendations: ['slow cooked yo ho ho saucy wench', 'crock pot meatball stew', 'slow cooker lentil beef stew', 'pork and cabbage stew', 'pollo crepes con mole azul', 'chili rice casserole', 'shortcut brunswick stew by campbell s', 'creamy bratwurst stew', 'quick fix beef stew', 'wowlers   runzas']
